<a href="https://colab.research.google.com/github/cintiapinho/logicaprogramacaopython/blob/main/Pequenos%20Talentos/Jogo_da_Velha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jogo da Velha

In [1]:
import numpy as np
import tkinter as tk
from tkinter import messagebox

In [6]:
import numpy as np

# Rede Neural para o jogo da velha
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate

        self.weights_input_hidden = np.random.randn(self.input_size, self.hidden_size)
        self.weights_hidden_output = np.random.randn(self.hidden_size, self.output_size)

        self.bias_hidden = np.random.randn(self.hidden_size)
        self.bias_output = np.random.randn(self.output_size)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward(self, X):
        self.hidden_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = self.sigmoid(self.hidden_input)

        self.final_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.final_output = self.sigmoid(self.final_input)

        return self.final_output

    def backpropagation(self, X, y, output):
        output_error = y - output
        output_delta = output_error * self.sigmoid_derivative(output)

        hidden_error = np.dot(output_delta, self.weights_hidden_output.T)
        hidden_delta = hidden_error * self.sigmoid_derivative(self.hidden_output)

        self.weights_hidden_output += np.dot(self.hidden_output.T, output_delta) * self.learning_rate
        self.weights_input_hidden += np.dot(X.T, hidden_delta) * self.learning_rate

        self.bias_output += np.sum(output_delta, axis=0) * self.learning_rate
        self.bias_hidden += np.sum(hidden_delta, axis=0) * self.learning_rate

    def train(self, X, y, iterations=10000):
        for _ in range(iterations):
            output = self.forward(X)
            self.backpropagation(X, y, output)

# Função para converter o estado do tabuleiro em vetor
def board_to_input(board):
    return np.array(board).flatten()

# Função para treinar a rede neural
def train_nn(nn):
    # Exemplo simples de treinamento
    training_inputs = np.array([
        board_to_input([1, 0, 0, 0, 0, 0, 0, 0, 0]),  # Jogador X em (0, 0)
        board_to_input([0, 1, 0, 0, 0, 0, 0, 0, 0]),  # Jogador X em (0, 1)
        board_to_input([0, 0, 1, 0, 0, 0, 0, 0, 0]),  # Jogador X em (0, 2)
    ])

    training_outputs = np.array([
        [0, 1, 0, 0, 0, 0, 0, 0, 0],  # Melhor movimento seria (0, 1)
        [0, 0, 1, 0, 0, 0, 0, 0, 0],  # Melhor movimento seria (0, 2)
        [0, 0, 0, 0, 1, 0, 0, 0, 0],  # Melhor movimento seria (1, 1)
    ])

    nn.train(training_inputs, training_outputs)

# Função para a IA escolher um movimento
def ai_move(board, nn):
    input_board = board_to_input(board)
    output = nn.forward(input_board)
    move = np.argmax(output)
    return move

# Simulação de jogo sem interface gráfica
class TicTacToeGame:
    def __init__(self):
        self.board = [0] * 9  # Tabuleiro inicial vazio
        self.current_player = 1  # 1 para jogador X, -1 para O
        self.nn = NeuralNetwork(9, 18, 9)  # Inicializa a rede neural

        train_nn(self.nn)  # Treina a rede neural antes do jogo começar

    def print_board(self):
        symbols = {1: 'X', -1: 'O', 0: ' '}
        print("\nBoard:")
        for i in range(3):
            print(" | ".join([symbols[self.board[i*3 + j]] for j in range(3)]))
            if i < 2:
                print("---------")

    def player_move(self, index):
        if self.board[index] == 0:  # Se a célula estiver vazia
            self.board[index] = self.current_player
            self.print_board()
            if self.check_winner(self.current_player):
                print("Você venceu!")
                return True
            elif 0 not in self.board:
                print("Empate!")
                return True
            else:
                self.current_player = -1
                return False

    def ai_turn(self):
        move = ai_move(self.board, self.nn)
        if self.board[move] == 0:
            self.board[move] = self.current_player
            self.print_board()
            if self.check_winner(self.current_player):
                print("A IA venceu!")
                return True
            elif 0 not in self.board:
                print("Empate!")
                return True
            else:
                self.current_player = 1
                return False

    def check_winner(self, player):
        win_conditions = [(0, 1, 2),
                          (3, 4, 5),
                          (6, 7, 8),
                          (0, 3, 6),
                          (1, 4, 7),
                          (2, 5, 8),
                          (0, 4, 8),
                          (2, 4, 6)]
        return any(all(self.board[pos] == player for pos in condition) for condition in win_conditions)

    def play_game(self):
        self.print_board()
        while True:
            move = int(input("Escolha sua posição (0-8): "))
            if self.player_move(move):
                break
            if self.ai_turn():
                break

# Executar o jogo sem interface gráfica
game = TicTacToeGame()
game.play_game()


Board:
  |   |  
---------
  |   |  
---------
  |   |  
Escolha sua posição (0-8): 0

Board:
X |   |  
---------
  |   |  
---------
  |   |  

Board:
X | O |  
---------
  |   |  
---------
  |   |  
Escolha sua posição (0-8): 4

Board:
X | O |  
---------
  | X |  
---------
  |   |  
Escolha sua posição (0-8): 8

Board:
X | O |  
---------
  | X |  
---------
  |   | O
Escolha sua posição (0-8): 2

Board:
X | O | O
---------
  | X |  
---------
  |   | O


KeyboardInterrupt: Interrupted by user